In [1]:
import collections
import pathlib

import pandas as pd
import spacy
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

BASE_DIR = pathlib.Path.cwd().parent.parent.resolve()
BASE_DIR

WindowsPath('C:/Users/Alejandro Feria/Desktop/Unal/8 semestre/pnl/SilkNews')

### Cargar dataset

In [2]:
clean_df = pd.read_csv(f"{BASE_DIR}/src/data/clean_data/noticias_combinadas.csv")
clean_df['text'] = clean_df['text'].astype(str)
print(clean_df.shape)

muestra = 500
muestras_df = clean_df.sample(muestra, random_state=42)
print(muestras_df.shape)

clean_df.head()

(5568, 2)
(500, 2)


,text,label
0,en una rueda de prensa las autoridades dijeron...,True
1,una operacion en la que las autoridades irland...,True
2,el nuevo informe se basa en mas de entrevistas...,True
3,las pruebas hechas a varias de las principales...,True
4,michelle y john wylie estuvieron buscando dura...,True


In [3]:
# uv run --with pip spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")
nlp

## tokenizar y aplicar stemming

In [4]:
stemmer = SnowballStemmer('spanish')
def tokenize(text: str) -> list[str]:
  return [stemmer.stem(token.text) for token in nlp(text) if not token.is_punct and not token.is_space]

muestras_df["tokens"] = muestras_df["text"].apply(tokenize)


In [5]:
muestras_df.head()

,text,label,tokens
1168,los socialistas reunen a cerca de cargos en un...,True,"[los, social, reun, a, cerc, de, carg, en, una..."
765,la candidata del pp a la comunidad de madrid i...,True,"[la, candidat, del, pp, a, la, comun, de, madr..."
465,con el objetivo de paliar las penurias de pers...,True,"[con, el, objet, de, pali, las, penuri, de, pe..."
1117,la desaceleracion economica y la normalizacion...,True,"[la, desaceler, econom, y, la, normaliz, del, ..."
5371,un fuerte miedo y temor experimento brayan per...,False,"[un, fuert, mied, y, temor, experiment, bray, ..."


## Crear BoW

In [6]:
countVectorizer = CountVectorizer(ngram_range=(1, 2))
x_bow = countVectorizer.fit_transform(muestras_df["tokens"].str.join(sep=" ").to_numpy())
print(countVectorizer.get_feature_names_out())
print(countVectorizer.vocabulary_)  # dict con el vocabulario

['aaiun' 'aaiun hub' 'abaj' ... 'ﬁnal de' 'ﬁrme' 'ﬁrme oposicion']
{'los': 29958, 'social': 46681, 'reun': 43545, 'cerc': 6394, 'de': 10535, 'carg': 5824, 'en': 17006, 'unas': 51429, 'jorn': 26708, 'trabaj': 49680, 'el': 15509, 'grup': 22940, 'la': 27181, 'diput': 14398, 'alic': 1853, 'reunir': 43563, 'hoy': 24569, 'municipal': 33848, 'alcald': 1597, 'portavoc': 39143, 'concejal': 8374, 'que': 41018, 'se': 44565, 'trat': 49963, 'los social': 30533, 'social reun': 46723, 'reun cerc': 43546, 'cerc de': 6395, 'de carg': 10798, 'carg en': 5833, 'en unas': 17520, 'unas jorn': 51446, 'jorn de': 26710, 'de trabaj': 12036, 'trabaj el': 49694, 'el grup': 15987, 'grup social': 22969, 'social en': 46695, 'en la': 17290, 'la diput': 27498, 'diput de': 14399, 'de alic': 10600, 'alic reunir': 1856, 'reunir hoy': 43565, 'hoy cerc': 24572, 'carg municipal': 5841, 'municipal alcald': 33849, 'alcald portavoc': 1602, 'portavoc concejal': 39144, 'concejal en': 8378, 'trabaj en': 49696, 'en el': 17179, 'el

# Sin stopwords

In [8]:
vocabulario_sin_stopwords = {}
# texto puede ser un unigrama o bigrama
for texto, frecuencia in countVectorizer.vocabulary_.items():
    palabras = texto.split()
    if any(palabra in nlp.Defaults.stop_words for palabra in palabras):
        texto_nuevo = " ".join([palabra for palabra in palabras if palabra not in nlp.Defaults.stop_words])
        if texto_nuevo == "":
            continue
    else:
        texto_nuevo = texto
    vocabulario_sin_stopwords[texto_nuevo] = frecuencia

print(vocabulario_sin_stopwords)

{'social': 46728, 'reun': 23278, 'cerc': 33957, 'carg': 5832, 'jorn': 26716, 'trabaj': 49707, 'grup': 22970, 'diput': 14405, 'alic': 17027, 'reunir': 43564, 'municipal': 33851, 'alcald': 15546, 'portavoc': 39143, 'concejal': 8379, 'trat': 19624, 'social reun': 46723, 'reun cerc': 43546, 'grup social': 22969, 'alic reunir': 1856, 'carg municipal': 5841, 'municipal alcald': 33849, 'alcald portavoc': 1602, 'portavoc concejal': 39144, 'candidat': 12634, 'pp': 39320, 'comun': 8025, 'madr': 30802, 'isabel': 26445, 'diaz': 14086, 'ayus': 4064, 'denunci': 13219, 'miercol': 12851, 'victim': 52560, 'campan': 17089, 'machist': 30784, 'delezn': 13056, 'hech': 23980, 'contr': 23704, 'aspir': 3475, 'gobern': 12779, 'region': 42845, 'sal': 8312, 'defens': 28548, 'vox': 41789, 'madr isabel': 30812, 'isabel diaz': 26446, 'diaz ayus': 14085, 'machist delezn': 30781, 'hech contr': 24002, 'aspir gobern': 3472, 'objet': 35409, 'pali': 36473, 'penuri': 37510, 'person': 37857, 'riesg': 31658, 'exclusion': 11

In [ ]:
bigrama = []
unigrama = []
for k, v in vocabulario_sin_stopwords.items():
    if " " in k:
        bigrama.append((k, v))
    else:
        unigrama.append((k, v))
len(unigrama), len(bigrama)

(7819, 16913)

## Frecuencia menor a 3

In [10]:
# si la frencuencia es menos de 3 eliminar palabras
all_tokens = [token for sublist in muestras_df["tokens"] for token in sublist]
token_counts = collections.Counter(all_tokens)
tokens_to_remove = {token for token, count in token_counts.items() if count < 3}

vocabulario_eliminacion_frecuencia = {palabra: indice for palabra, indice in countVectorizer.vocabulary_.items() if palabra not in tokens_to_remove}
print(vocabulario_eliminacion_frecuencia)

{'los': 29958, 'social': 46681, 'reun': 43545, 'cerc': 6394, 'de': 10535, 'carg': 5824, 'en': 17006, 'unas': 51429, 'jorn': 26708, 'trabaj': 49680, 'el': 15509, 'grup': 22940, 'la': 27181, 'diput': 14398, 'alic': 1853, 'hoy': 24569, 'municipal': 33848, 'alcald': 1597, 'concejal': 8374, 'que': 41018, 'se': 44565, 'trat': 49963, 'los social': 30533, 'social reun': 46723, 'reun cerc': 43546, 'cerc de': 6395, 'de carg': 10798, 'carg en': 5833, 'en unas': 17520, 'unas jorn': 51446, 'jorn de': 26710, 'de trabaj': 12036, 'trabaj el': 49694, 'el grup': 15987, 'grup social': 22969, 'social en': 46695, 'en la': 17290, 'la diput': 27498, 'diput de': 14399, 'de alic': 10600, 'alic reunir': 1856, 'reunir hoy': 43565, 'hoy cerc': 24572, 'carg municipal': 5841, 'municipal alcald': 33849, 'alcald portavoc': 1602, 'portavoc concejal': 39144, 'concejal en': 8378, 'trabaj en': 49696, 'en el': 17179, 'el que': 16288, 'que se': 41665, 'se trat': 45009, 'candidat': 5589, 'del': 12569, 'pp': 39314, 'comun': 

### Si en el bigrama hay una palabra con frecuencia menor a 3, eliminar palabra

In [11]:
print("Cantidad de palabras en el vocabulario original:", len(countVectorizer.vocabulary_))
print("Cantidad de palabras en el vocabulario sin stopwords:", len(vocabulario_sin_stopwords))
print("Cantidad de palabras en el vocabulario con eliminación de frecuencia:", len(vocabulario_eliminacion_frecuencia))

Cantidad de palabras en el vocabulario original: 53592
Cantidad de palabras en el vocabulario sin stopwords: 24732
Cantidad de palabras en el vocabulario con eliminación de frecuencia: 48816


In [12]:
bigrama = []
unigrama = []
for k, v in vocabulario_sin_stopwords.items():
    if " " in k:
        bigrama.append((k, v))
    else:
        unigrama.append((k, v))

In [13]:
len(unigrama), len(bigrama)

(7819, 16913)

In [14]:
print(bigrama)

[('social reun', 46723), ('reun cerc', 43546), ('grup social', 22969), ('alic reunir', 1856), ('carg municipal', 5841), ('municipal alcald', 33849), ('alcald portavoc', 1602), ('portavoc concejal', 39144), ('madr isabel', 30812), ('isabel diaz', 26446), ('diaz ayus', 14085), ('machist delezn', 30781), ('hech contr', 24002), ('aspir gobern', 3472), ('exclusion social', 20316), ('social carit', 46687), ('carit castellon', 5874), ('program activ', 40142), ('activ lleg', 424), ('lleg durant', 29564), ('durant beneficiari', 15117), ('poblacion person', 38404), ('person logr', 37872), ('desaceler econom', 13366), ('merc inmobiliari', 32418), ('empez hac', 16916), ('hac mell', 23543), ('ladrill despu', 28364), ('vari anos', 52042), ('recuper bast', 42631), ('bast mir', 4345), ('ultim mes', 50458), ('ultim trimestr', 50476), ('ejempl ent', 15463), ('fuert mied', 21995), ('mied temor', 32733), ('temor experiment', 48731), ('experiment bray', 20461), ('bray perez', 4905), ('anos asalt', 2485), (